In [1]:
import pandas as pd
import json
import re
import csv
import pyTigerGraph as tg
from sklearn.model_selection import train_test_split


# Notebook to explore and generate a graph for the Domain Record datasets

* Mangle data to make loading into the graph easier
* Instantiate the graph schema and graph queries (using TigerGraph - www.tigergraph.com )
* Perform community detection
* Collect community statistics to be used as features


In [2]:
!wc -l "../data/processed/better_whois_data.csv" 

    9344 ../data/processed/better_whois_data.csv


In [3]:
# benign_whois_data.txt

!wc -l "../data/raw/benign_whois_data.txt" 


    4000 ../data/raw/benign_whois_data.txt


In [4]:
# Process the benign data file - row by row processing of json / dictionary file - Cumbersome but cautious

b = '../data/raw/benign_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 4001


In [5]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [6]:
# Look for and process any errors -  There should now be none 
# (initial stages of project there were some errors in the data)

error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [7]:
benign_df = pd.DataFrame.from_dict(record_dict, orient='index')
benign_df.reset_index(inplace=True)

In [8]:
benign_df.head(3)

,index,domain_name,name_servers,registrant_contact_name,registrant_name,registrar,status,updated_date,admin_c,billing_c,...,registrant_zip,billing_id,billing_phone_number,billing_postal_code,billing_state_province,admin_country_code,registrar_country_code,registrar_organization_id,registrar_postal_code,registrar_street
0,griffithspartners.com.au,COM.AU,"[T.AU, R.AU, Q.AU, S.AU]",CEO,None,Afilias Australia Pty Ltd,[serverDeleteProhibited https://afilias.com.au...,2022-03-17 14:34:56,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gestaoconcurso.com.br,None,NaN,NaN,None,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21fss.com,"[21FSS.COM, 21fss.com]","[NS33.DOMAINCONTROL.COM, NS34.DOMAINCONTROL.COM]",NaN,NaN,"GoDaddy.com, LLC",[clientDeleteProhibited https://icann.org/epp#...,"[2021-01-19 22:26:36, 2021-01-10 00:51:38]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
!wc -l "../data/raw/malicious_whois_data.txt" 

    2415 ../data/raw/malicious_whois_data.txt


In [10]:
# Process the malicious data set (just like we did the benign dataset)

b = '../data/raw/malicious_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 2415


In [11]:
# create the dictionary with the data
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [12]:
# Process any errors

error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [13]:
mal_df = pd.DataFrame.from_dict(record_dict, orient='index')
mal_df.reset_index(inplace=True)

In [14]:
mal_df.head(3)

,index,address,city,country,creation_date,dnssec,domain_name,emails,expiration_date,name,...,tech_phone_ext,registrar_zip_code,registrant_type,admin_account_name,admin_company_name,admin_family_name,registrant_company_name,tech_account_name,tech_company_name,tech_family_name
0,reseptors.com,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,KN,"[2021-12-08 08:53:26, 2021-12-08T08:53:26]",unsigned,RESEPTORS.COM,domainabuse@tucows.com,"[2022-12-08 08:53:26, 2022-12-08T08:53:26]",REDACTED FOR PRIVACY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,contentcdns.net,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,KN,"[2022-02-17 23:18:15, 2022-02-17T23:18:15]",unsigned,CONTENTCDNS.NET,domainabuse@tucows.com,"[2023-02-17 23:18:15, 2023-02-17T23:18:15]",REDACTED FOR PRIVACY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,izocab.com,"220013, Belarus, Minsk, ul.YA.Kolasa, d.31, kv...",Minsk,BY,2015-12-30 00:30:35,"[unsigned, Unsigned]",IZOCAB.COM,"[abuse@reg.ru, info@brale.ru]",2022-12-30 00:30:35,Vladimir Nikolskii,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#########

In [15]:
mal_df['malicious'] = 1
benign_df['malicious'] = 0

In [16]:
combined_df = pd.concat([mal_df,benign_df], axis=0)

In [17]:
len(combined_df['index'])

6342

In [18]:
random_seed = 0
primary_dataset = pd.read_csv('../data/processed/better_whois_data.csv', low_memory=False) #6510 rows
y_col = 'malicious'
y = primary_dataset[y_col]
X = primary_dataset[primary_dataset.columns.drop(y_col)]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

print(f"Total Rows: {len(primary_dataset)}")
print(f"Train rows: {len(X_train)}")

Total Rows: 9128
Train rows: 7302


In [19]:
DomainRecord_cols = ['index', 'dnssec', 'name', 'malicious']

combined_df.loc[:,DomainRecord_cols].to_csv("../data/external/combined_whois_data.csv")
combined_df.loc[combined_df['index'].isin(X_train['domain']),DomainRecord_cols].to_csv("../data/external/combined_whois_data_train.csv")
combined_df.loc[combined_df['index'].isin(X_test['domain']),DomainRecord_cols].to_csv("../data/external/combined_whois_data_test.csv")

In [20]:
list_of_explode_columns = ['country', 'emails','whois_server', 'domain_status', 'registrar', 
                           'name_servers']

In [21]:
for _col in list_of_explode_columns:
    combined_df.loc[:,['index',_col]].explode(_col).to_csv(
    f"../data/external/combined_whois_data_{_col}.csv")
    
    combined_df.loc[combined_df['index'].isin(X_train['domain']),['index',_col]].explode(_col).to_csv(
    f"../data/external/combined_whois_data_{_col}_train.csv")
    
    combined_df.loc[combined_df['index'].isin(X_test['domain']),['index',_col]].explode(_col).to_csv(
    f"../data/external/combined_whois_data_{_col}_test.csv")
    
    file1 = f"../data/external/combined_whois_data_{_col}.csv"
    file2 = f"../data/external/combined_whois_data_{_col}_train.csv"
    file3 = f"../data/external/combined_whois_data_{_col}_test.csv"
    
    !wc -l $file1 $file2 $file3

    6367 ../data/external/combined_whois_data_country.csv
    5513 ../data/external/combined_whois_data_country_train.csv
    1690 ../data/external/combined_whois_data_country_test.csv
   13570 total
    9319 ../data/external/combined_whois_data_emails.csv
    8091 ../data/external/combined_whois_data_emails_train.csv
    2450 ../data/external/combined_whois_data_emails_test.csv
   19860 total
    6343 ../data/external/combined_whois_data_whois_server.csv
    5494 ../data/external/combined_whois_data_whois_server_train.csv
    1684 ../data/external/combined_whois_data_whois_server_test.csv
   13521 total
    6358 ../data/external/combined_whois_data_domain_status.csv
    5508 ../data/external/combined_whois_data_domain_status_train.csv
    1686 ../data/external/combined_whois_data_domain_status_test.csv
   13552 total
    6343 ../data/external/combined_whois_data_registrar.csv
    5494 ../data/external/combined_whois_data_registrar_train.csv
    1684 ../data/external/combined_whois_dat

In [22]:
### Entropy data import

In [23]:
# Read in the file
with open('../data/raw/benign_entropy_data.txt', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('{', '')
filedata = filedata.replace('}', '')
filedata = filedata.replace(':', ',')

# Write the file out again
with open('../data/external/benign_entropy_data.txt', 'w') as file:
  file.write(filedata)

entropy_train = []
entropy_test = []

for f in filedata.split("\n"):
    domain = f[1:f.find('"', 1)]
    if domain in X_train['domain'].values:
        entropy_train.append(f)
    else:
        entropy_test.append(f)
        
with open('../data/external/benign_entropy_data_train.txt', 'w') as file:
    for item in entropy_train:
        file.write("%s\n" % item)
        
with open('../data/external/benign_entropy_data_test.txt', 'w') as file:
    for item in entropy_test:
        file.write("%s\n" % item)
        
file1 = '../data/external/benign_entropy_data.txt'
file2 = '../data/external/benign_entropy_data_train.txt'
file3 = '../data/external/benign_entropy_data_test.txt'

!wc -l $file1 $file2 $file3

    4001 ../data/external/benign_entropy_data.txt
    3572 ../data/external/benign_entropy_data_train.txt
     430 ../data/external/benign_entropy_data_test.txt
    8003 total


In [24]:
# Read in the file
with open('../data/raw/malicious_entropy_data.txt', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('{', '')
filedata = filedata.replace('}', '')
filedata = filedata.replace(':', ',')

# Write the file out again
with open('../data/external/malicious_entropy_data.txt', 'w') as file:
  file.write(filedata)

entropy_train = []
entropy_test = []

for f in filedata.split("\n"):
    domain = f[1:f.find('"', 1)]
    if domain in X_train['domain'].values:
        entropy_train.append(f)
    else:
        entropy_test.append(f)

with open('../data/external/malicious_entropy_data_train.txt', 'w') as file:
    for item in entropy_train:
        file.write("%s\n" % item)
        
with open('../data/external/malicious_entropy_data_test.txt', 'w') as file:
    for item in entropy_test:
        file.write("%s\n" % item)
        
file1 = '../data/external/malicious_entropy_data.txt'
file2 = '../data/external/malicious_entropy_data_train.txt'
file3 = '../data/external/malicious_entropy_data_test.txt'

!wc -l $file1 $file2 $file3

    2415 ../data/external/malicious_entropy_data.txt
    1941 ../data/external/malicious_entropy_data_train.txt
     475 ../data/external/malicious_entropy_data_test.txt
    4831 total


In [25]:
### Load IP Address Org details

import pandas as pd
import json

b = '../data/raw/benign_ip_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")

Number of line errors: 0
Number of lines: 6670


In [26]:
b = '../data/raw/malicious_ip_data.txt'

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")

Number of line errors: 0
Number of lines: 9085


In [27]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [28]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [29]:
org_df = pd.DataFrame.from_dict(record_dict, orient='index')
org_df.reset_index(inplace=True)

In [30]:
org_df['A_Org'] = org_df['A'].apply(pd.Series)['Org']
org_df['MX_Org'] = org_df['MX'].apply(pd.Series)['Org']

In [31]:
org_df.sample(10)

,index,A,MX,A_Org,MX_Org
5630,www.gbsjttwdmxqvurwymtuw.com,"{'CC': 'RU', 'Org': 'REGRU-MNT'}","{'CC': 'NA', 'Org': 'NA'}",REGRU-MNT,NA
2087,facebook.ca,"{'CC': 'US', 'Org': 'Facebook, Inc.'}","{'CC': 'NA', 'Org': 'NA'}","Facebook, Inc.",NA
78,lejournaldusiecle.com,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
6242,togoleruno.site,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
325,lex-score.com,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
6113,surroundbest.top,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
5686,bfrvmquxmaxbcdrrqfrg.com,"{'CC': 'RU', 'Org': 'REGRU-MNT'}","{'CC': 'NA', 'Org': 'NA'}",REGRU-MNT,NA
872,endofanear.com,"{'CC': 'US', 'Org': 'Squarespace, Inc.'}","{'CC': 'US', 'Org': 'pair Networks'}","Squarespace, Inc.",pair Networks
3230,momsdreamjob.com,"{'CC': 'US', 'Org': 'Contract Fourteen - MDR'}","{'CC': 'NA', 'Org': 'NA'}",Contract Fourteen - MDR,NA
1473,krauthammer.com,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA


In [32]:
org_df.loc[org_df['MX_Org']!="NA",["index","MX_Org"]].to_csv("../data/external/combined_mx_org.csv")
org_df.loc[(org_df['MX_Org']!="NA") & (org_df['index'].isin(X_train['domain'])) ,["index","MX_Org"]].to_csv("../data/external/combined_mx_org_train.csv")
org_df.loc[(org_df['MX_Org']!="NA") & (org_df['index'].isin(X_test['domain'])),["index","MX_Org"]].to_csv("../data/external/combined_mx_org_test.csv")

org_df.loc[org_df['A_Org']!="NA",["index","A_Org"]].to_csv("../data/external/combined_A_org.csv")
org_df.loc[(org_df['MX_Org']!="NA") & (org_df['index'].isin(X_train['domain'])),["index","A_Org"]].to_csv("../data/external/combined_A_org_train.csv")
org_df.loc[(org_df['MX_Org']!="NA") & (org_df['index'].isin(X_train['domain'])),["index","A_Org"]].to_csv("../data/external/combined_A_org_test.csv")

file1 = "../data/external/combined_mx_org.csv"
file2 = "../data/external/combined_mx_org_train.csv"
file3 = "../data/external/combined_mx_org_test.csv"

!wc -l $file1 $file2 $file3

file1 = "../data/external/combined_A_org.csv"
file2 = "../data/external/combined_A_org_train.csv"
file3 = "../data/external/combined_A_org_test.csv"

!wc -l $file1 $file2 $file3

    2387 ../data/external/combined_mx_org.csv
    2158 ../data/external/combined_mx_org_train.csv
     713 ../data/external/combined_mx_org_test.csv
    5258 total
    4158 ../data/external/combined_A_org.csv
    2158 ../data/external/combined_A_org_train.csv
    2158 ../data/external/combined_A_org_test.csv
    8474 total


In [33]:
!docker ps

CONTAINER ID   IMAGE                                     COMMAND                  CREATED       STATUS         PORTS                                                                                                                               NAMES
38e46715231e   docker.tigergraph.com/tigergraph:latest   "/bin/sh -c '/usr/sb…"   3 weeks ago   Up 4 minutes   0.0.0.0:9000->9000/tcp, :::9000->9000/tcp, 0.0.0.0:14240->14240/tcp, :::14240->14240/tcp, 0.0.0.0:14022->22/tcp, :::14022->22/tcp   tigergraph


In [34]:
# To start tigergraph docker
# https://docs.tigergraph.com/tigergraph-server/current/getting-started/docker
#  docker run -d -p 14022:22 -p 9000:9000 -p 14240:14240 --name tigergraph --ulimit nofile=1000000:1000000 -v /Users/nb311848/Documents/mystuff/repos/artemis/data/external:/home/tigergraph/data -t docker.tigergraph.com/tigergraph:latest


In [113]:
conn = tg.TigerGraphConnection()


In [114]:
# print(conn.gsql('CLEAR GRAPH STORE -HARD'))

In [115]:
# Clear the Tigergraph server/graph as we want to reproduce the graph creation from scratch
## CAUTION: this wipes everything inside your TigerGraph container !!

print(conn.gsql('drop all', options=[]))

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph Artemis, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE and GSE
Stopping GPE GSE
Successfully stopped GPE GSE in 0.005 seconds
Clearing graph store...
Successfully cleared graph store
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.086 seconds
Everything is dropped.


In [116]:
# Read in the GSQL code that creates the Graph / Schema / Loading Jobs and Queries on TigerGraph

text_file = open("../src/scripts/DBImportExport_Artemis.gsql", "r")
 
#read whole file to a string
artemis_graph_gsql = text_file.read()
 
#close file
text_file.close()
 
# print(artemis_graph_gsql)

In [117]:
# Run the GSQL script and print the results (Check for any errors)

print(conn.gsql(artemis_graph_gsql, options=[]))

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 30.539 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.083 seconds
The graph Artemis is created.
Successfully created schema change jobs: [create_artemis_graph].

Current graph version 0
Trying to add vertex DomainRecord.
Trying to add vertex DomainName.
Trying to add vertex Country.
Trying to add vertex City.
Trying to add vertex Emails.
Trying to add vertex Organisation.
Trying to add vertex Nameserver.
Trying to add vertex Registrar.
Trying to add vertex DomainRecordStatus.
Trying to add vertex WhoisServer.
Trying to add edge DomainRecord_DomainName.
Trying to add edge Country_City.
Trying to add edge DomainRecord_Country.
Trying to add edge DomainRecord_Emails.
Trying to add edge DomainRecord_Organisation.
Trying to add edge DomainRecord_Nameserver.
Trying to add edge DomainRecord_Registrar.
Trying to add edge DomainRecord_DomainRecordStatus.
Trying to add edge DomainRecord_WhoisServer.
Trying t

In [118]:
print(conn.gsql('INSTALL QUERY ALL', options=[]))

Start installing queries, about 1 minute ...
delete_co_edges query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/delete_co_edges'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
tg_louvain query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/tg_louvain?v_type=VALUE&e_type=VALUE&[wt_attr=VALUE]&[max_iter=VALUE]&[result_attr=VALUE]&[file_path=VALUE]&[print_info=VALUE]'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
community_stuff query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/community_stuff'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
tg_label_prop query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/tg_label_prop?v_type=VALUE&e_type=VALUE&max_iter=VALUE&output_limit=VALUE&[print_accum=VALUE]&[file_path=VALUE]&[attr=VALUE]'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
community_features_calc query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/community_features_calc'. Add -H "

In [145]:
# set train / test switch
train_test = 'test'

In [146]:
# Generic loading job string

loading_job = """RUN LOADING JOB load_job_whois_data{field1} USING MyDataSource="/home/tigergraph/data/combined_whois_data{field2}.csv\""""

In [147]:
# Execute the loading job string for the listed data files



print(conn.gsql(loading_job.format(field1=f"",field2=f"_{train_test}"),graphname='Artemis', options=[]))

for x in list_of_explode_columns:
    print(conn.gsql(loading_job.format(field1=f"_{x}",field2=f"_{x}_{train_test}"),graphname='Artemis', options=[]))                       




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_whois_data, jobid: Artemis.load_job_whois_data.file.m1.1650114214669
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_whois_data.file.m1.1650114214669.log'

Job "Artemis.load_job_whois_data.file.m1.1650114214669" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_whois_data.file.m1.1650114214669" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+----------------------------------------------------------------------------------------------+
|                                          FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/combined_whois_data_test.csv |           1684 |     16 kl/s |     0.10 s|
+-----------

In [148]:
# Load entropy data
loading_job = f'RUN LOADING JOB load_job_entropy USING MyDataSource="/home/tigergraph/data/benign_entropy_data_{train_test}.txt"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))

loading_job = f'RUN LOADING JOB load_job_entropy USING MyDataSource="/home/tigergraph/data/malicious_entropy_data_{train_test}.txt"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_entropy, jobid: Artemis.load_job_entropy.file.m1.1650114252415
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_entropy.file.m1.1650114252415.log'

Job "Artemis.load_job_entropy.file.m1.1650114252415" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_entropy.file.m1.1650114252415" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+----------------------------------------------------------------------------------------------+
|                                          FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/benign_entropy_data_test.txt |            430 |      4 kl/s |     0.10 s|
+--------------------------

In [149]:
# Load MX / A org data
loading_job = f'RUN LOADING JOB load_job_A_org USING MyDataSource="/home/tigergraph/data/combined_A_org_{train_test}.csv"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))

loading_job = f'RUN LOADING JOB load_job_MX_org USING MyDataSource="/home/tigergraph/data/combined_MX_org_{train_test}.csv"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_A_org, jobid: Artemis.load_job_A_org.file.m1.1650114263545
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_A_org.file.m1.1650114263545.log'

Job "Artemis.load_job_A_org.file.m1.1650114263545" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_A_org.file.m1.1650114263545" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+-----------------------------------------------------------------------------------------+
|                                     FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/combined_A_org_test.csv |           2158 |     18 kl/s |     0.12 s|
+---------------------------------------------------

In [150]:
conn.graphname = 'Artemis'

In [151]:
# Run co_edge creation

print(conn.runInstalledQuery("community_stuff", timeout = 30000))
print(conn.runInstalledQuery("delete_co_loop_edges", timeout = 30000))

[]
[]


In [152]:
# run label propogation

# tg_label_prop (SET<STRING> v_type, SET<STRING> e_type, INT max_iter, INT output_limit, 
#  BOOL print_accum = TRUE, STRING file_path = "", STRING attr = "") 

# have to use query strings until pyTigerGrpah allows lists for sets

# lfp_new_nodes_label_prop

params = "v_type=DomainRecord&e_type=co_registrar&e_type=c_org&e_type=co_nameserver&max_iter=10000&output_limit=0&print_accum=1&attr=community"

if train_test=="train":
    result = conn.runInstalledQuery("tg_label_prop", params=params, timeout = 30000)
    community_features = conn.runInstalledQuery("community_features_calc", timeout = 30000)
    community_features = conn.runInstalledQuery("community_features_calc", timeout = 30000)
else:
    result = conn.runInstalledQuery("lfp_new_nodes_label_prop", params=params, timeout = 30000)

In [153]:
if train_test=="train":
    community_features_df = pd.DataFrame(community_features[0]['(@@group_entropy_final)'])
    community_features_df.to_csv("../data/processed/community_features.csv")
else:
    community_features_df = pd.read_csv("../data/processed/community_features.csv")

In [154]:
community_features_df.sample(3)

,Unnamed: 0,community,min_entropy,max_entropy,avg_entropy,domain_count,malicious_ratio
54,54,13631516,2.92193,3.12193,3.04655,3,0.00000
116,116,14680227,1.92193,3.39275,2.80562,14,0.00000
94,94,19922989,1.37095,4.32320,3.57795,199,0.97487


In [155]:
len(community_features_df)

142

In [137]:
DomainRecordsGSQL = """INTERPRET QUERY () FOR GRAPH Artemis {
   t = select dr from DomainRecord:dr;
   print(t);
}"""

domain_records = conn.runInterpretedQuery(DomainRecordsGSQL)


In [138]:
dr_dict = {}

for x in domain_records[0]['(t)']:
    dr_dict[x['v_id']] = x['attributes']['community']

In [139]:
domain_record_df = pd.DataFrame.from_dict(dr_dict, orient='index').reset_index()
domain_record_df.columns = ['DomainRecord','community']

In [141]:
domain_record_df.sample(3)

,DomainRecord,community
740,animemax.buzz,16777326
3058,legifrance.org,-1
2980,credithousehold1998b.com,23068723


In [142]:
graph_features_df = domain_record_df.merge(community_features_df, how = 'left', on='community')

In [143]:
graph_features_df.sample(3)

,DomainRecord,community,min_entropy,max_entropy,avg_entropy,domain_count,malicious_ratio
5098,gramha.net,32505946,0.00000,3.77356,2.78828,446,0.63453
3084,baiyunshan.com.cn,-1,0.00000,4.02895,2.82822,1029,0.31681
1656,gxhhzn.tech,25165842,1.79248,3.32486,2.50353,135,0.07407


In [144]:
graph_features_df.to_csv(f'../data/processed/graph_community_features_{train_test}.csv', index=False)